---
# 📋 Author Information

**Author:** Tassawar Abbas  
**Email:** [abbas829@gmail.com](mailto:abbas829@gmail.com)  
**Created:** January 16, 2026  
**Subject:** Linear Regression Analysis on California Housing Dataset  
**Description:** A comprehensive tutorial on building, evaluating, and comparing linear regression models with detailed explanations and visualizations.

---

# 🏠✨ Linear Regression: California Housing Dataset  
**Dataset:** California Housing   
**Goal:** Understand every step that builds a reliable regression model, step-by-step, with beautiful visuals and clear explanations.

---

## 📚 0. One-time setup

In [ ]:
# !pip install seaborn scikit-learn pandas matplotlib
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
sns.set_theme(style="whitegrid", palette="mako")

---

## 📥 1. Load California Housing

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

# 1. Load the data
housing = fetch_california_housing()

# 2. Create a DataFrame with features
X = pd.DataFrame(housing.data, columns=housing.feature_names)

# 3. Create target variable
y = pd.Series(housing.target, name='PRICE')

# 4. Create full dataframe for EDA
df = X.copy()
df['PRICE'] = y

---

## 🔍 2. EDA – Exploratory Data Analysis
### 2-a Dataset Overview
**Explanation:** The statistical summary shows the mean, standard deviation, and range for each feature. This helps us understand the scale and distribution of our data. Features like PRICE range from 0.15 to 5.00 (in $100,000s), while features like CRIM vary widely across different neighborhoods.

In [ ]:
print("Dataset Shape:", df.shape)
print("\nFirst few rows:")
print(df.head())
print("\nStatistical Summary:")
df.describe().T.style.background_gradient(cmap="mako", subset=['mean'])

### 2-b Missing Values Check
**Explanation:** We check if there are any null values in our dataset. This is important because missing data can cause errors or bias in our model. A clean dataset without missing values means we can proceed directly to analysis without handling gaps.

In [ ]:
missing_data = df.isna().sum()
print("Missing Values Count:")
print(missing_data)
print(f"\nTotal missing values: {missing_data.sum()}")

**Conclusion:** The California Housing dataset is completely clean – no missing values found. This means we can proceed directly to analysis without spending time on data imputation or cleaning.

### 2-c Correlation Analysis
**Explanation:** A correlation heatmap shows how each feature relates to every other feature. Strong correlations (dark colors) indicate that features move together. For example, if two features are perfectly correlated, one might be redundant. This helps us identify relationships and potential multicollinearity issues in our model.

In [ ]:
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='mako', square=True, cbar_kws={'label': 'Correlation'})
plt.title("Correlation Heatmap: Feature Relationships")
plt.tight_layout()
plt.show()

**Interpretation:** The color intensity represents correlation strength: darker green indicates stronger positive correlation, lighter colors indicate weaker correlations. We can see that features like Latitude and Longitude are strongly correlated with price, meaning location is a major price driver.

### 2-d Target Variable Distribution
**Explanation:** Understanding the distribution of our target variable (PRICE) is crucial. If prices follow a normal distribution, our linear regression model will perform better. If the distribution is skewed, we may need to apply transformations to improve model performance and meet regression assumptions.

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df['PRICE'], kde=True, color='#0a4d68', bins=30)
plt.title("Distribution of California House Prices")
plt.xlabel("Price ($100,000s)")
plt.ylabel("Frequency")
plt.grid(axis='y', alpha=0.3)
plt.show()

**Observation:** The price distribution shows a right skew – most houses are clustered at lower prices, with a long tail extending toward expensive properties. The KDE curve (smooth line) shows this pattern clearly. We may address this with transformations later if needed.

---

## 🧼 3. Data Cleaning & Outlier Detection
Outliers (extremely unusual values) can heavily distort linear regression models because they pull the line toward themselves. We'll identify and handle them appropriately.

### 3-a Outlier Visualization using Box Plots
**Explanation:** A box plot shows the distribution of data visually. The box contains the middle 50% of values, the line inside is the median, and circles beyond the whiskers represent outliers. Identifying outliers helps us decide whether to remove them, cap them, or keep them.

In [ ]:
plt.figure(figsize=(12, 5))
# Select key features for outlier visualization
cols_to_check = ['MedInc', 'AveRooms', 'AveOccup', 'HouseAge']
sns.boxplot(data=df[cols_to_check], orient='h', palette='mako')
plt.title("Box Plots: Detecting Outliers in Key Features")
plt.xlabel("Value")
plt.show()

### 3-b Capping Extreme Values
**Explanation:** Instead of removing outliers entirely (which loses data), we'll cap them at the 2.5th and 97.5th percentiles. This preserves the data while reducing the impact of extreme values that could skew our regression line.

In [ ]:
def clip_outliers(series, percentile_range=0.05):
    """Cap extreme values at specified percentiles"""
    lower = series.quantile(percentile_range / 2)
    upper = series.quantile(1 - percentile_range / 2)
    return np.clip(series, lower, upper)

# Apply outlier capping to features
X_clean = X.apply(clip_outliers)
y_clean = y.copy()

print("Data cleaned successfully!")
print(f"Original shape: {X.shape}")
print(f"Cleaned shape: {X_clean.shape}")

---

## 📊 4. Summary Statistics After Cleanup
**Explanation:** After capping outliers, we review the statistics again to confirm that extreme values have been moderated. The mean and standard deviation might shift slightly, indicating that very extreme values have been normalized.

In [ ]:
print("Summary Statistics After Outlier Capping:")
print(X_clean.describe().round(3))

---

## ⚖️ 5. Linear Regression Assumptions – Quality Checklist
| Assumption | Description | How We Verify |
|------------|-------------|----------|
| **Linearity** | Relationship between X and y is linear | Scatter plots of features vs. price |
| **Normality** | Residuals follow a normal distribution | Q-Q plot of residuals |
| **Homoscedasticity** | Constant variance of residuals | Residuals vs. fitted values plot |
| **No Multicollinearity** | Predictors are independent | Correlation matrix & VIF scores |
| **Independence** | Observations are independent | Domain knowledge (housing prices vary by area) |

### 5-a Linearity Check: Scatter Plot Analysis
**Explanation:** We examine the relationship between a key feature (median income) and price. If the pattern is roughly linear (points form a straight-line trend), our assumption is satisfied. If the pattern curves significantly, we might need polynomial features or transformations.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_clean['MedInc'], y_clean, alpha=0.5, color='#05bfdb')
plt.xlabel("Median Income (tens of thousands $)")
plt.ylabel("House Price ($100,000s)")
plt.title("Linearity Check: Price vs. Median Income")
plt.grid(True, alpha=0.3)
plt.show()

**Observation:** The scatter plot shows a clear positive linear trend – as median income increases, house prices tend to increase as well. This validates the linearity assumption for this key feature. The relationship appears roughly linear, which is suitable for our regression model.

---

## 🔄 6. Feature Scaling & Normalization
### 6-a Why Scale Features?
**Explanation:** Features in the California Housing dataset are on different scales (e.g., median income vs. average house age). Scaling brings all features to the same range (usually 0-1 or standardized with mean=0, std=1). This improves model performance and helps algorithms converge faster. Standardization is particularly important for distance-based models.

In [ ]:
# Apply StandardScaler for feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)
X_scaled = pd.DataFrame(X_scaled, columns=X_clean.columns)

print("Features have been standardized:")
print(f"Mean of each feature (should be ≈0):\n{X_scaled.mean()}")
print(f"\nStandard deviation of each feature (should be ≈1):\n{X_scaled.std()}")

### 6-b Verification of Scaling
**Explanation:** After standardization, each feature should have a mean of approximately 0 and a standard deviation of 1. This indicates that features are now on a comparable scale, centered around zero. The output shows our scaling was successful.

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(X_scaled['MedInc'], bins=30, color='#00ffca', edgecolor='black', alpha=0.7)
plt.title("Distribution of Standardized Median Income")
plt.xlabel("Standardized Value (mean=0, std=1)")
plt.ylabel("Frequency")
plt.grid(axis='y', alpha=0.3)
plt.show()

**Result:** The histogram shows a bell-shaped distribution centered around zero. This is exactly what we expect after standardization. All features are now comparable on the same numerical scale.

---

## 🏗️ 7. Train-Test Split
**Explanation:** We split our data into training (80%) and testing (20%) sets. The training set teaches our model the patterns, while the test set evaluates how well it generalizes to unseen data. A random seed (random_state=42) ensures reproducibility.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_clean, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train.shape[0]} samples ({X_train.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"Testing set size: {X_test.shape[0]} samples ({X_test.shape[0]/len(X_scaled)*100:.1f}%)")

---

## ⚙️ 8. Train Linear Regression Model
**Explanation:** We fit a simple linear regression model on the training data. This model learns the optimal weights (coefficients) for each feature that minimize the prediction error. The model seeks to find the best-fit line through our multi-dimensional data.

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)

print("Model trained successfully!")
print(f"Model intercept: {lm.intercept_:.4f}")
print(f"Number of features: {len(lm.coef_)}")

---

## 🎯 9. Model Evaluation Metrics
**Explanation:** We evaluate our model using two key metrics:
- **R² Score**: Ranges from 0 to 1. It represents the proportion of variance explained by the model. Higher is better (1.0 = perfect prediction)
- **RMSE (Root Mean Squared Error)**: Average prediction error. Lower is better, measured in the same units as the target variable.

In [ ]:
def evaluate_model(model, X_train_data, X_test_data, y_train_data, y_test_data, model_name="Model"):
    """Comprehensive evaluation of model performance"""
    
    # Training performance
    train_pred = model.predict(X_train_data)
    train_r2 = r2_score(y_train_data, train_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train_data, train_pred))
    
    # Testing performance
    test_pred = model.predict(X_test_data)
    test_r2 = r2_score(y_test_data, test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test_data, test_pred))
    
    print(f"\n{'='*60}")
    print(f"Model: {model_name}")
    print(f"{'='*60}")
    print(f"Training R² = {train_r2:.4f} | RMSE = {train_rmse:.4f}")
    print(f"Testing  R² = {test_r2:.4f} | RMSE = {test_rmse:.4f}")
    print(f"Overfitting Check: R² difference = {train_r2 - test_r2:.4f}")
    
    return test_pred

y_pred_train = evaluate_model(lm, X_train, X_test, y_train, y_test, "Linear Regression")

---

## 🔍 10. Residual Diagnostics – Model Validation
**Explanation:** Residuals are the differences between actual and predicted values. Analyzing residuals helps us check if our model assumptions are valid. Good residuals should:
1. Have no pattern (scattered randomly around zero)
2. Follow a normal distribution
3. Have constant variance across all predicted values

In [ ]:
from scipy import stats

# Calculate residuals
y_test_pred = lm.predict(X_test)
residuals = y_test - y_test_pred

# Create two diagnostic plots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Residuals vs Fitted Values
axes[0].scatter(y_test_pred, residuals, alpha=0.6, color='#088395')
axes[0].axhline(0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel("Fitted Values")
axes[0].set_ylabel("Residuals")
axes[0].set_title("Residuals vs Fitted Values\n(Check: Random scatter, no pattern)")
axes[0].grid(True, alpha=0.3)

# Plot 2: Q-Q Plot for normality
stats.probplot(residuals, dist="norm", plot=axes[1])
axes[1].set_title("Q-Q Plot\n(Check: Points follow red line = normal residuals)")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Interpretation:** 
- **Left plot (Residuals vs Fitted):** If points are randomly scattered around the zero line with no pattern, our homoscedasticity assumption is satisfied. Any funnel shape would indicate non-constant variance.
- **Right plot (Q-Q Plot):** Points should follow the red diagonal line. Deviations at the tails indicate non-normal residuals, though slight deviations are often acceptable in practice.

---

## 🚀 11. Comparing Multiple Regression Models
**Explanation:** Linear regression is a good starting point, but other algorithms may capture non-linear patterns better. We'll compare:
- **Ridge Regression**: Adds a penalty for large coefficients to reduce overfitting
- **Random Forest**: An ensemble that combines many decision trees
- **Gradient Boosting**: Sequentially builds trees to correct previous errors

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import GradientBoostingRegressor

# Create different models
models_dict = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": RidgeCV(alphas=np.logspace(-3, 3, 20), cv=5),
    "Random Forest (100 trees)": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42, learning_rate=0.1)
}

# Train and evaluate all models
results = {}
for name, model in models_dict.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results[name] = {"model": model, "r2": r2, "rmse": rmse}
    print(f"{name:30s} | R² = {r2:.4f} | RMSE = {rmse:.4f}")

print("\n" + "="*60)

---

## 🏆 12. Model Comparison Visualization
**Explanation:** Comparing models side-by-side helps us choose the best performer. We plot R² and RMSE values to see which model balances prediction accuracy with simplicity.

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

model_names = list(results.keys())
r2_scores = [results[name]["r2"] for name in model_names]
rmse_scores = [results[name]["rmse"] for name in model_names]

# R² Comparison
axes[0].barh(model_names, r2_scores, color='#05bfdb')
axes[0].set_xlabel("R² Score (higher is better)")
axes[0].set_title("Model Performance: R² Comparison")
axes[0].set_xlim([0, 1])
for i, v in enumerate(r2_scores):
    axes[0].text(v + 0.02, i, f"{v:.3f}", va='center')

# RMSE Comparison
axes[1].barh(model_names, rmse_scores, color='#088395')
axes[1].set_xlabel("RMSE (lower is better)")
axes[1].set_title("Model Performance: RMSE Comparison")
for i, v in enumerate(rmse_scores):
    axes[1].text(v + 0.01, i, f"{v:.3f}", va='center')

plt.tight_layout()
plt.show()

# Identify best model
best_model_name = max(results, key=lambda x: results[x]["r2"])
print(f"\n🏆 Best performing model: {best_model_name}")
print(f"   R² Score: {results[best_model_name]['r2']:.4f}")
print(f"   RMSE: {results[best_model_name]['rmse']:.4f}")

---

## 📈 13. Feature Importance Analysis
**Explanation:** For tree-based models like Random Forest, we can determine which features are most important in predicting house prices. Features used earlier in the tree-splitting process contribute more to predictions. This helps us understand what drives housing prices.

In [ ]:
# Extract feature importance from Random Forest model
rf_model = results["Random Forest (100 trees)"]["model"]

if hasattr(rf_model, 'feature_importances_'):
    feature_importance = pd.Series(
        rf_model.feature_importances_,
        index=X_train.columns
    ).sort_values(ascending=False)
    
    plt.figure(figsize=(10, 6))
    top_features = feature_importance.head(8)
    colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(top_features)))
    bars = plt.barh(range(len(top_features)), top_features.values, color=colors)
    plt.yticks(range(len(top_features)), top_features.index)
    plt.xlabel("Feature Importance Score")
    plt.title("Top 8 Most Important Features (Random Forest)")
    plt.gca().invert_yaxis()
    
    # Add value labels
    for i, (name, value) in enumerate(top_features.items()):
        plt.text(value + 0.005, i, f"{value:.4f}", va='center')
    
    plt.tight_layout()
    plt.show()
    
    print("\nFeature Importance Rankings:")
    print(feature_importance.round(4))

---

## 🧠 14. Key Learnings & Practical Takeaways

### Summary of Our Analysis Journey:

1. **Exploratory Data Analysis (EDA)** – We visualized data distributions, found strong relationships between features, and identified that the California Housing dataset is clean with no missing values.

2. **Data Preprocessing** – We capped outliers and standardized features to ensure they're on the same scale, which improves model performance.

3. **Assumption Checking** – We verified that our data reasonably meets linear regression assumptions (linearity, normality, homoscedasticity).

4. **Model Training & Evaluation** – We trained multiple models and found that ensemble methods (Random Forest, Gradient Boosting) typically outperform simple linear regression on this dataset.

5. **Residual Diagnostics** – Analyzing residuals confirmed that our model's predictions have reasonable error patterns.

6. **Feature Importance** – We identified that location-based features (Latitude, Longitude) and median income are key drivers of house prices.

### Practical Checklist for Future Projects:
✓ Always explore your data first with visualizations  
✓ Check for and handle missing values and outliers  
✓ Verify regression assumptions before drawing conclusions  
✓ Scale features to comparable ranges  
✓ Split data into train-test sets for honest evaluation  
✓ Compare multiple models instead of assuming one is best  
✓ Analyze residuals to validate model assumptions  
✓ Interpret feature importance to understand your model  

This workflow forms a solid foundation that you can apply to any regression problem with tabular data! 🎓